# Reddit Scrapper
- This notebook contains all the steps required to connect with Reddit's API and get data from any given subreddit
- Credentials and tokens have been erased for security reasons

In [1]:
#This piece of code will allow us to authenticate against Reddit's API
personal_use = 'PERSONAL_USE_TOKEN'
secret ='SECRET_TOKEN'
import requests
auth = requests.auth.HTTPBasicAuth(personal_use,secret)
data = {
    'grant_type':'client_credentials',
    'username':'YOUR_EMAIL_GOES HERE',
    'password':'YOUR_PASSWORD_GOES_HERE'
}
headers = {
    'User-Agent':'WHATEVR'
}
res = requests.post('https://www.reddit.com/api/v1/access_token', auth=auth, data=data, headers=headers)
token = res.json()['access_token']
headers['Authorization'] = f'bearer {token}'
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

<Response [200]>

In [15]:
#This cell defines the dataframe structure, queries te API and transforms the results into a manageable dataframe
import pandas as pd

data_dict ={
    'titles' : [],
    'posts' : [],
    'subreddit': [],
    'creation_date':[],
    'authors' : [],
    'upvotes':[],
    'url':[]
    
}

api = 'https://oauth.reddit.com'
params ={'limit':100}
counter=0
while True:
    if counter==50:
        break
    res = requests.get(f"{api}/r/wallstreetbets/new", headers=headers, params=params)
    res = res.json()
    counter+=1
    for posting in res['data']['children']:
        data_dict['titles'].append(posting['data']['title'])
        data_dict['posts'].append(posting['data']['selftext'])
        data_dict['subreddit'].append(posting['data']['subreddit'])
        data_dict['creation_date'].append(posting['data']['created_utc'])
        data_dict['authors'].append(posting['data']['author_fullname'])
        data_dict['upvotes'].append(posting['data']['ups'])
        data_dict['url'].append(posting['data']['url'])

df = pd.DataFrame(data_dict)
df.head()


,titles,posts,subreddit,creation_date,authors,upvotes,url
0,Big Meta shareholders rage at 'tone-deaf' Mark...,,wallstreetbets,1.667246e+09,t2_b0m3haio,1,https://nypost.com/2022/10/31/big-meta-shareho...
1,GME in 15k shares. Last call for this torpedo!,,wallstreetbets,1.667246e+09,t2_a0wqpu0h,15,https://i.redd.it/w4ul4jmn37x91.jpg
2,"Remarks of Richard B. Smith, Commissioner Unit...","""What I want to discuss with you today is a pi...",wallstreetbets,1.667245e+09,t2_2a5jbp59,1,https://www.reddit.com/r/wallstreetbets/commen...
3,Where's my God Damn $55.20 Elon?,,wallstreetbets,1.667245e+09,t2_e7q5y,7,https://i.redd.it/iq1bdb1427x91.jpg
4,Down another 6% today!,,wallstreetbets,1.667245e+09,t2_7d81jt8g,33,https://i.redd.it/x4gfb7aa07x91.png


In [16]:
len(df)

5000

In [17]:
#We delete empty posts as these are useless for the scope of the project
df=df[df['posts']!=""]
df.head()

,titles,posts,subreddit,creation_date,authors,upvotes,url
2,"Remarks of Richard B. Smith, Commissioner Unit...","""What I want to discuss with you today is a pi...",wallstreetbets,1.667245e+09,t2_2a5jbp59,1,https://www.reddit.com/r/wallstreetbets/commen...
5,Brazil,Brazil just finished their run-off election wi...,wallstreetbets,1.667244e+09,t2_6n2z4j2y,2,https://www.reddit.com/r/wallstreetbets/commen...
7,Gamelancer Media Corp. $GMNG $GAMGF has a new ...,Gamelancer Media Corp. has uploaded a new corp...,wallstreetbets,1.667243e+09,t2_a1jf7gbk,3,https://www.reddit.com/r/wallstreetbets/commen...
8,Trading SPY and FOMC Meeting,"Hey guys, this is another big week for the fut...",wallstreetbets,1.667243e+09,t2_5ylhcs6t,8,https://www.reddit.com/r/wallstreetbets/commen...
9,$LLY earnings call,"$LLY earnings, boomer play.\n\nMy 405c FDs for...",wallstreetbets,1.667243e+09,t2_gq4rwz6a,0,https://www.reddit.com/r/wallstreetbets/commen...


In [18]:
len(df)

2000

In [19]:
#Just checking what some rows contain
df.iloc[1]['posts']

"Brazil just finished their run-off election with Lula the former president beating the incumbent in the closest election in Brazil's democratic victory and the incumbent Jair Bolsonaro has been parroting Trump leading up to the election and has yet to concede. He was a former military leader with the support of the military and in the past has spread the sentiment that he isn't leaving. I wanted exposure to the instability of the country so I bought puts in an ETF there for after the transition of power in the new year. The Brazilian market is generally up today on Lula's victory but I think the instability is not priced in. I chose the ETF EWZ as it has a higher finance exposure which should be the first sector to hurt. I see at least some instability and at most a Coup d'état. Happy Profiting on suffering welcome to capitalism.\n\n&amp;#x200B;\n\nPositions  Jan 20 23   Put. $17 x20, $21x29, $25x11, $29x2"

In [20]:
#Export the final dataframe to a csv for later use
df.to_csv('WallStreetBets.csv',index=False)